# Import dependencies

In [1]:
import os
import sys

sys.path.insert(0, os.path.dirname(os.getcwd())) 

In [2]:
import time
import gc

import numpy as np
import pandas as pd

from transformers import BertTokenizer, BertModel
from transformers import logging

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

from semeval_reader import SemevalReader

from InputDataset import InputDataset

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from torch import cuda

from sa_models.SA_BERT_Dropout_Linear import SA_BERT_Dropout_Linear

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
logging.set_verbosity_error() 

In [4]:
print(torch.cuda.get_device_name(0))
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory // 1024 ** 3} GB")

NVIDIA GeForce RTX 2060 SUPER
Memory: 8 GB


In [5]:
def clear_memory():
    torch.cuda.empty_cache()

    with torch.no_grad():
        torch.cuda.empty_cache()

    gc.collect()

# Load Data

In [6]:
def get_target_list_for_polarity(polarity):
    if polarity == 'positive':
        return [0, 0, 1]
    if polarity == 'negative':
        return [1, 0, 0]
    return [0, 1, 0]

In [7]:
semeval_reader = SemevalReader('../../../data/semeval16_restaurants_train.xml')

reviews = semeval_reader.read_reviews()
absolute_polarity_sentences = semeval_reader.get_absolute_polarity_sentences()

df = pd.DataFrame(map(lambda x: (x.text, x.opinions[0].polarity), absolute_polarity_sentences))
df.rename(columns={0: 'text'}, inplace=True)
df['target_list'] = df.apply(lambda row: get_target_list_for_polarity(row[1]), axis=1)

absolute_polarity_df = df.drop(columns=[1])

# Train & Validate

In [8]:
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4

EPOCHS = 2

LEARNING_RATE = 1e-5

TRAIN_SPLIT = 0.8

NO_RUNS = 10

In [9]:
BERT_FINE_TUNED_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_fine_tuned.pth'

In [10]:
MODEL_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/models/bert_pre_trained_dropout_linear.pth'
STATS_OUTPUT = '../../../results/SA/SemEval16 - Task 5 - Restaurants/stats/bert_pre_trained_dropout_linear.csv'

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [12]:
best_accuracy = 0.0

In [13]:
def train(epoch, model, loss_fn, optimizer, dataloader):
    model.train()

    dataloader_len = len(dataloader)

    for _,data in enumerate(dataloader, 0):
        optimizer.zero_grad()

        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        
        if _ % (dataloader_len // 10) == 0:
            print(f"Epoch: {epoch}/{EPOCHS}, Batch: {_}/{dataloader_len}, Loss: {loss.item()}")
        
        loss.backward()
        
        optimizer.step()

In [14]:
def validation(model, dataloader):
    model.eval()
    
    fin_targets=[]
    fin_outputs=[]

    with torch.no_grad():
        for _, data in enumerate(dataloader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)

            outputs = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_targets

In [15]:
results = pd.DataFrame(columns=['accuracy','precision_score_micro','precision_score_macro','recall_score_micro','recall_score_macro','f1_score_micro','f1_score_macro', 'execution_time'])

In [16]:
for i in range(10):
    # clear cache cuda
    torch.cuda.empty_cache()
    with torch.no_grad():
        torch.cuda.empty_cache()
    gc.collect()

    start_time = time.time()

    print(f"Run {i + 1}/10")

    train_dataset = df.sample(frac=TRAIN_SPLIT)
    test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    training_set = InputDataset(train_dataset, tokenizer)
    testing_set = InputDataset(test_dataset, tokenizer)

    train_dataloader = DataLoader(
        training_set,
        sampler = RandomSampler(train_dataset),
        batch_size = TRAIN_BATCH_SIZE,
        drop_last = True
    )

    validation_dataloader = DataLoader(
        testing_set,
        sampler = SequentialSampler(testing_set),
        batch_size = VALID_BATCH_SIZE,
        drop_last = True
    )

    model = SA_BERT_Dropout_Linear(BertModel.from_pretrained('bert-base-uncased'), dropout=0.3, no_out_labels=3, device=device).to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    for epoch in range(EPOCHS):
        train(epoch, model, loss_fn, optimizer, train_dataloader)

    outputs, targets = validation(model, validation_dataloader)
    outputs = np.argmax(outputs, axis=1)
    targets = np.argmax(targets, axis=1)
    
    accuracy = accuracy_score(targets, outputs)
    precision_score_micro = precision_score(targets, outputs, average='micro')
    precision_score_macro = precision_score(targets, outputs, average='macro')
    recall_score_micro = recall_score(targets, outputs, average='micro')
    recall_score_macro = recall_score(targets, outputs, average='macro')
    f1_score_micro = f1_score(targets, outputs, average='micro')
    f1_score_macro = f1_score(targets, outputs, average='macro')

    execution_time = time.time() - start_time

    results.loc[i] = [accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro, execution_time]

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.bert, BERT_FINE_TUNED_OUTPUT)
        torch.save(model, MODEL_OUTPUT)

    del train_dataset
    del test_dataset
    del training_set
    del testing_set
    del model
    del loss_fn
    del optimizer
    del outputs
    del targets

Run 1/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6745090484619141
Epoch: 0/2, Batch: 31/313, Loss: 0.4020112454891205
Epoch: 0/2, Batch: 62/313, Loss: 0.664030909538269
Epoch: 0/2, Batch: 93/313, Loss: 0.3989991545677185
Epoch: 0/2, Batch: 124/313, Loss: 0.18994998931884766
Epoch: 0/2, Batch: 155/313, Loss: 0.4249964654445648
Epoch: 0/2, Batch: 186/313, Loss: 0.3951959013938904
Epoch: 0/2, Batch: 217/313, Loss: 0.08335362374782562
Epoch: 0/2, Batch: 248/313, Loss: 0.10956870019435883
Epoch: 0/2, Batch: 279/313, Loss: 0.11274321377277374
Epoch: 0/2, Batch: 310/313, Loss: 0.13868239521980286
Epoch: 1/2, Batch: 0/313, Loss: 0.11591733992099762
Epoch: 1/2, Batch: 31/313, Loss: 0.1295178383588791
Epoch: 1/2, Batch: 62/313, Loss: 0.3271807134151459
Epoch: 1/2, Batch: 93/313, Loss: 0.05405610799789429
Epoch: 1/2, Batch: 124/313, Loss: 0.04850679636001587
Epoch: 1/2, Batch: 155/313, Loss: 0.052392616868019104
Epoch: 1/2, Batch: 186/313, Loss: 0.03831210732460022
Epoch: 1/2, Batch: 217/313, Loss: 0.0

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 2/10
Epoch: 0/2, Batch: 0/313, Loss: 0.6438738703727722
Epoch: 0/2, Batch: 31/313, Loss: 0.5249814987182617
Epoch: 0/2, Batch: 62/313, Loss: 0.4618469178676605
Epoch: 0/2, Batch: 93/313, Loss: 0.6665202379226685
Epoch: 0/2, Batch: 124/313, Loss: 0.4468272626399994
Epoch: 0/2, Batch: 155/313, Loss: 0.35689350962638855
Epoch: 0/2, Batch: 186/313, Loss: 0.2591734528541565
Epoch: 0/2, Batch: 217/313, Loss: 0.3624417185783386
Epoch: 0/2, Batch: 248/313, Loss: 0.19417956471443176
Epoch: 0/2, Batch: 279/313, Loss: 0.12640480697155
Epoch: 0/2, Batch: 310/313, Loss: 0.06427465379238129
Epoch: 1/2, Batch: 0/313, Loss: 0.15176278352737427
Epoch: 1/2, Batch: 31/313, Loss: 0.08965463936328888
Epoch: 1/2, Batch: 62/313, Loss: 0.35068389773368835
Epoch: 1/2, Batch: 93/313, Loss: 0.049426764249801636
Epoch: 1/2, Batch: 124/313, Loss: 0.4027048945426941
Epoch: 1/2, Batch: 155/313, Loss: 0.06668153405189514
Epoch: 1/2, Batch: 186/313, Loss: 0.5164370536804199
Epoch: 1/2, Batch: 217/313, Loss: 0.0434

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 3/10
Epoch: 0/2, Batch: 0/313, Loss: 0.7471268177032471
Epoch: 0/2, Batch: 31/313, Loss: 0.5450891256332397
Epoch: 0/2, Batch: 62/313, Loss: 0.5376306772232056
Epoch: 0/2, Batch: 93/313, Loss: 0.29744479060173035
Epoch: 0/2, Batch: 124/313, Loss: 0.3270938992500305
Epoch: 0/2, Batch: 155/313, Loss: 0.5500168204307556
Epoch: 0/2, Batch: 186/313, Loss: 0.09070420265197754
Epoch: 0/2, Batch: 217/313, Loss: 0.12451867759227753
Epoch: 0/2, Batch: 248/313, Loss: 0.05633945390582085
Epoch: 0/2, Batch: 279/313, Loss: 0.14092348515987396
Epoch: 0/2, Batch: 310/313, Loss: 0.41968482732772827
Epoch: 1/2, Batch: 0/313, Loss: 0.09486141055822372
Epoch: 1/2, Batch: 31/313, Loss: 0.10085143148899078
Epoch: 1/2, Batch: 62/313, Loss: 0.06692194938659668
Epoch: 1/2, Batch: 93/313, Loss: 0.21786019206047058
Epoch: 1/2, Batch: 124/313, Loss: 0.15292593836784363
Epoch: 1/2, Batch: 155/313, Loss: 0.08337284624576569
Epoch: 1/2, Batch: 186/313, Loss: 0.04874255508184433
Epoch: 1/2, Batch: 217/313, Loss: 

...

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 9/10
Epoch: 0/2, Batch: 0/313, Loss: 0.8291071653366089
Epoch: 0/2, Batch: 31/313, Loss: 0.6433000564575195
Epoch: 0/2, Batch: 62/313, Loss: 0.38120850920677185
Epoch: 0/2, Batch: 93/313, Loss: 0.3563120365142822
Epoch: 0/2, Batch: 124/313, Loss: 0.3736670911312103
Epoch: 0/2, Batch: 155/313, Loss: 0.3583146035671234
Epoch: 0/2, Batch: 186/313, Loss: 0.4057040214538574
Epoch: 0/2, Batch: 217/313, Loss: 0.29249581694602966
Epoch: 0/2, Batch: 248/313, Loss: 0.25716280937194824
Epoch: 0/2, Batch: 279/313, Loss: 0.38166946172714233
Epoch: 0/2, Batch: 310/313, Loss: 0.12228326499462128
Epoch: 1/2, Batch: 0/313, Loss: 0.18159125745296478
Epoch: 1/2, Batch: 31/313, Loss: 0.15389476716518402
Epoch: 1/2, Batch: 62/313, Loss: 0.15118065476417542
Epoch: 1/2, Batch: 93/313, Loss: 0.39233654737472534
Epoch: 1/2, Batch: 124/313, Loss: 0.11781491339206696
Epoch: 1/2, Batch: 155/313, Loss: 0.10082535445690155
Epoch: 1/2, Batch: 186/313, Loss: 0.409903347492218
Epoch: 1/2, Batch: 217/313, Loss: 0.1

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Run 10/10
Epoch: 0/2, Batch: 0/313, Loss: 0.8231021761894226
Epoch: 0/2, Batch: 31/313, Loss: 0.6095651388168335
Epoch: 0/2, Batch: 62/313, Loss: 0.49971359968185425
Epoch: 0/2, Batch: 93/313, Loss: 0.4496971368789673
Epoch: 0/2, Batch: 124/313, Loss: 0.41186952590942383
Epoch: 0/2, Batch: 155/313, Loss: 0.31987789273262024
Epoch: 0/2, Batch: 186/313, Loss: 0.49088382720947266
Epoch: 0/2, Batch: 217/313, Loss: 0.39992642402648926
Epoch: 0/2, Batch: 248/313, Loss: 0.3761046826839447
Epoch: 0/2, Batch: 279/313, Loss: 0.15212470293045044
Epoch: 0/2, Batch: 310/313, Loss: 0.15819814801216125
Epoch: 1/2, Batch: 0/313, Loss: 0.16191069781780243
Epoch: 1/2, Batch: 31/313, Loss: 0.1084735095500946
Epoch: 1/2, Batch: 62/313, Loss: 0.2785627245903015
Epoch: 1/2, Batch: 93/313, Loss: 0.26461392641067505
Epoch: 1/2, Batch: 124/313, Loss: 0.12940561771392822
Epoch: 1/2, Batch: 155/313, Loss: 0.3989400267601013
Epoch: 1/2, Batch: 186/313, Loss: 0.49926209449768066
Epoch: 1/2, Batch: 217/313, Loss: 0

D:\Apps\Anaconda\envs\ml\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
results

,accuracy,precision_score_micro,precision_score_macro,recall_score_micro,recall_score_macro,f1_score_micro,f1_score_macro,execution_time
0,0.894231,0.894231,0.597084,0.894231,0.591830,0.894231,0.592882,206.694469
1,0.849359,0.849359,0.556800,0.849359,0.603103,0.849359,0.568444,197.642759
2,0.907051,0.907051,0.595715,0.907051,0.626766,0.907051,0.610431,199.676465
3,0.881410,0.881410,0.586310,0.881410,0.593438,0.881410,0.589028,198.570202
4,0.868590,0.868590,0.572546,0.868590,0.568046,0.868590,0.568607,198.737928
5,0.849359,0.849359,0.548030,0.849359,0.599074,0.849359,0.568938,198.109338
6,0.894231,0.894231,0.581734,0.894231,0.614052,0.894231,0.596886,197.536018
7,0.913462,0.913462,0.605322,0.913462,0.614630,0.913462,0.609851,198.161038
8,0.900641,0.900641,0.588424,0.900641,0.622112,0.900641,0.603145,197.508012
9,0.871795,0.871795,0.569354,0.871795,0.598549,0.871795,0.583549,197.437127


In [18]:
results.to_csv(STATS_OUTPUT)